In [6]:
!pip install diffusers transformers torch python-dotenv accelerate matplotlib

  Using cached matplotlib-3.9.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached contourpy-1.2.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.53.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (162 kB)
  Using cached kiwisolver-1.4.5-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.4 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
Using cached matplotlib-3.9.0-cp310-cp310-macosx_11_0_arm64.whl (7.8 MB)
Using cached contourpy-1.2.1-cp310-cp310-macosx_11_0_arm64.whl (244 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.53.0-cp310-cp310-macosx_11_0_arm64.whl (2.2 MB)
Using cached kiwisolver-1.4.5-cp310-cp310-macosx_11_0_arm64.whl (66 kB)
Using cached pyparsing-3.1.2-py3-none-any.whl (103 kB)


In [7]:
import os
from dotenv import load_dotenv
from huggingface_hub import login

load_dotenv()

login(token=os.getenv("HF_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/nick/.cache/huggingface/token
Login successful


In [ ]:
# Note you need to get permission to access this model from HF - https://huggingface.co/stabilityai/stable-diffusion-3-medium

In [8]:
import os
from transformers import T5EncoderModel, BitsAndBytesConfig
from diffusers import StableDiffusion3Pipeline
import torch
from PIL import Image
import matplotlib.pyplot as plt

# Configure quantization to load the model in 8-bit precision
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the text encoder model with quantization
text_encoder = T5EncoderModel.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers",
    subfolder="text_encoder_3",
    quantization_config=quantization_config,
)

# Load the Stable Diffusion pipeline with the default text encoder
pipe_default = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers",
    device_map="balanced",
    torch_dtype=torch.float16
)

# Load the Stable Diffusion pipeline with the custom text encoder
pipe_custom = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers",
    text_encoder_3=text_encoder,
    device_map="balanced",
    torch_dtype=torch.float16
)

# Define the prompt and generation parameters
prompt = "a photo of a cat holding a sign that says hello world"
negative_prompt = ""
num_inference_steps = 28
height = 1024
width = 1024
guidance_scale = 7.0

# Generate an image with the default text encoder
image_default = pipe_default(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=num_inference_steps,
    height=height,
    width=width,
    guidance_scale=guidance_scale,
).images[0]

# Generate an image with the custom text encoder
image_custom = pipe_custom(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=num_inference_steps,
    height=height,
    width=width,
    guidance_scale=guidance_scale,
).images[0]

# Save the images
image_default_path = "sd3_hello_world_default.png"
image_custom_path = "sd3_hello_world_custom.png"
image_default.save(image_default_path)
image_custom.save(image_custom_path)

# Display the images in the Jupyter notebook
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

# Display default text encoder image
axs[0].imshow(image_default)
axs[0].axis('off')
axs[0].set_title("Default Text Encoder")

# Display custom text encoder image
axs[1].imshow(image_custom)
axs[1].axis('off')
axs[1].set_title("Custom Text Encoder")

plt.show()


RuntimeError: No GPU found. A GPU is needed for quantization.